In [2]:
from typing import List
from abc import abstractmethod, ABC

class Action:
    pass

class InitAction(Action):
    def __init__(self, arr: List[int]):
        self.arr = [x for x in arr]

    def __repr__(self):
        return f'Init({self.arr})'

class CompareAction(Action):
    def __init__(self, i: int , j: int):
        self.i = i
        self.j = j

    def __repr__(self):
        return f'Compare({self.i}, {self.j})'

class SwapAction(Action):
    def __init__(self, i: int , j: int):
        self.i = i
        self.j = j

    def __repr__(self):
        return f'Swap({self.i}, {self.j})'


In [3]:
class SortingAlgorithm(ABC):
    def __init__(self, arr: List[int]):
        self.arr = arr
        self.actions: List[Action] = [InitAction(arr)]

    def _cmp(self, i: int, j: int) -> int:
        self.actions.append(CompareAction(i, j))
        return self.arr[i] - self.arr[j]

    def _swap(self, i: int, j: int) -> None:
        self.arr[i], self.arr[j] = self.arr[j], self.arr[i]
        self.actions.append(SwapAction(i, j))

    @abstractmethod
    def sort(self) -> None:
        raise NotImplemented()

    def getActions(self) -> List[Action]:
        return self.actions

In [4]:
class BubbleSort(SortingAlgorithm):
    def sort(self):
        for i in range(len(self.arr)):
            for j in range(i + 1, len(self.arr)):
                if self._cmp(i, j) > 0:
                    self._swap(i, j)


In [5]:
class MergeSort(SortingAlgorithm):
    def sort(self):
        self.innerSort(0, len(self.arr) - 1)

    def innerSort(self, l, r):
        if l < r:
            m = l + (r - l) // 2
            self.innerSort(l, m)
            self.innerSort(m + 1, r)
            self.merge(l, m, r)

    def merge(self, l, m, r):
        n1 = m - l + 1
        n2 = r - m

        # create temp arrays
        L = []
        R = []
        # Copy data to temp arrays L[] and R[]
        for i in range(0, n1):
            L.append({
                "value": self.arr[l + i],
                "index": l + i
            })

        for j in range(0, n2):
            R.append({
                "value": self.arr[m + 1 + j],
                "index": m + 1 + j
            })

        # Merge the temp arrays back into arr[l..r]
        i = 0     # Initial index of first subarray
        j = 0     # Initial index of second subarray
        k = l     # Initial index of merged subarray

        while i < n1 and j < n2:
            if self._cmp(L[i]["index"], R[j]["index"]) <= 0:
                if k != L[i]["index"]:
                    for item in L + R:
                        if item["value"] == self.arr[k]:
                            item["index"] = L[i]["index"]
                    self._swap(k, L[i]["index"])
                    L[i]["index"] = k
                i += 1
            else:
                if k != R[j]["index"]:
                    for item in L + R:
                        if item["value"] == self.arr[k]:
                            item["index"] = R[j]["index"]
                    self._swap(k, R[j]["index"])
                    R[j]["index"] = k
                j += 1
            k += 1

        # Copy the remaining elements of L[], if there
        # are any
        while i < n1:
            if k != L[i]["index"]:
                self._swap(k, L[i]["index"])
            i += 1
            k += 1

        # Copy the remaining elements of R[], if there
        # are any
        while j < n2:
            if k != R[j]["index"]:
                self._swap(k, R[j]["index"])
            j += 1
            k += 1


In [6]:
class QuickSort(SortingAlgorithm):
    def sort(self):
        self.innerSort(0, len(self.arr) - 1)

    def partition(self, low, high):
    # Choose the rightmost element as pivot
        # Pointer for greater element
        i = low - 1

        # Traverse through all elements
        # compare each element with pivot
        for j in range(low, high):
            if self._cmp(j, high) <= 0:

                # If element smaller than pivot is found
                # swap it with the greater element pointed by i
                i += 1

                # Swapping element at i with element at j
                self._swap(i, j)

        # Swap the pivot element with
        # the greater element specified by i
        self._swap(i + 1, high)

        # Return the position from where partition is done
        return i + 1

    def innerSort(self, low, high):
        if low < high:

            # Find pivot element such that
            # element smaller than pivot are on the left
            # element greater than pivot are on the right
            pi = self.partition(low, high)

            # Recursive call on the left of pivot
            self.innerSort(low, pi - 1)

            # Recursive call on the right of pivot
            self.innerSort(pi + 1, high)

# This code is contributed by Adnan Aliakbar

In [7]:
import matplotlib.pyplot as plt
from pathlib import Path

def createFrames(location: Path, sortAlg: SortingAlgorithm) -> None:
    if not location.exists():
        location.mkdir(parents=True, exist_ok=True)

    states = []
    for action in sortAlg.getActions():
        colors = ['tab:blue'] * len(sortAlg.arr)
        if isinstance(action, InitAction):
            states.append([x for x in action.arr])
        elif isinstance(action, SwapAction):
            prevState = states[-1]
            states.append([x for x in prevState])
            # Perform the swap on the last state
            states[-1][action.i], states[-1][action.j] = states[-1][action.j], states[-1][action.i]
            colors[action.i] = 'tab:red'
            colors[action.j] = 'tab:red'
        elif isinstance(action, CompareAction):
            prevState = states[-1]
            states.append([x for x in prevState])
            colors[action.i] = 'tab:red'
            colors[action.j] = 'tab:red'


        plt.bar(range(len(states[-1])), states[-1], color=colors)
        plt.axis('off')
        plt.savefig(f'{location}/frame_{len(states)}.png')
        plt.clf()


In [9]:
import os
import cv2

def generate_video(image_folder, video_output,  fps) -> None:
    if not Path(video_output).parent.exists():
        Path(video_output).parent.mkdir(parents=True, exist_ok=True)

    images = [img for img in sorted(os.listdir(image_folder), key=lambda x: int(x.split("_")[1].split(".")[0]))]
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(
        video_output,
        cv2.VideoWriter_fourcc(*'mp4v'),
        fps,
        (width, height)
    )
    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))

    cv2.destroyAllWindows()
    video.release()

ModuleNotFoundError: No module named 'cv2'

In [ ]:
class SortFramework():
    def __init__(self, alg, image_path, video_path, fps):
        self.alg = alg
        self.image_path = image_path
        self.video_path = video_path
        self.fps = fps

    def generate(self):
        self.alg.sort()
        createFrames(Path(self.image_path), self.alg)
        generate_video(self.image_path, self.video_path, self.fps)

In [ ]:
arr = [10,17,16,7,2,5,3,12,8,11,13,14,1,4,15,6,9,19,18]
bubble = BubbleSort([x for x in arr])
sortFramework = SortFramework(bubble, "output/bubble", "output/video/bubble.mp4", 30)
sortFramework.generate()
display.Video("output/video/bubble.mp4", embed=True)

In [ ]:
arr = [10,17,16,7,2,5,3,12,8,11,13,14,1,4,15,6,9,19,18]
merge = MergeSort([x for x in arr])
sortFramework = SortFramework(merge, "output/merge", "output/video/merge.mp4", 30)
sortFramework.generate()
display.Video("output/video/merge.mp4", embed=True)


NameError: name 'MergeSort' is not defined

In [ ]:
arr = [10,17,16,7,2,5,3,12,8,11,13,14,1,4,15,6,9,19,18]
quick = QuickSort([x for x in arr])
sortFramework = SortFramework(quick, "output/quick", "output/video/quick.mp4", 30)
sortFramework.generate()
display.Video("output/video/quick.mp4", embed=True)